# Data Stack Preparation (data-prep)

This tutorial will walk you through the workflow of the [VegMapper](https://github.com/NaiaraSPinto/VegMapper) repo. At the end of this tutorial, you will create multi-band geotiffs that can be used for the identification and classification of specific agroforestry systems, such as palm-oil plantations.

## 1) Get credentials ##

1) NASA Earthdata: https://urs.earthdata.nasa.gov/users/new

2) JAXA: https://www.eorc.jaxa.jp/ALOS/en/palsar_fnf/registration.htm

3) AWS S3/EC2: https://portal.aws.amazon.com/billing/signup#/start

4) Google Earth Engine: https://earthengine.google.com/

One-time authentication for Google Earth Engine:

In [2]:
import ee
ee.Authenticate()

Enter verification code: 


Exception: Problem requesting tokens. Please try again.  HTTP Error 400: Bad Request b'{\n  "error": "invalid_request",\n  "error_description": "Missing required parameter: code"\n}'

## 2) Set up data-prep conda environment ##

To create **data-prep** environment and install required packages:

In [15]:
!conda env create -f data-prep-env.yml

Solving environment: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use
#
#     $ conda activate data-prep
#
# To deactivate an active environment, use
#
#     $ conda deactivate



DEBUG menuinst_win32:__init__(200): Menu: name: 'Anaconda${PY_VER} ${PLATFORM}', prefix: 'D:\anaconda3\envs\data-prep', env_name: 'data-prep', mode: 'user', used_mode: 'user'
DEBUG menuinst_win32:create(324): Shortcut cmd is D:\anaconda3\python.exe, args are ['D:\\anaconda3\\cwp.py', 'D:\\anaconda3\\envs\\data-prep', 'D:\\anaconda3\\envs\\data-prep\\Library\\bin\\bash.exe', '--login', '-i', '--']


Activate **data-prep** environment:

In [36]:
!conda activate data-prep

Run **setup.py** to verify environment and permissions of scripts:

In [4]:
!python setup.py

ALOS-2\alos2_download_mosaic.py


Traceback (most recent call last):
  File "C:\Users\remyw\OneDrive - California Polytechnic State University\HDD\cal poly\classes\deep gis\palm oil\VegMapper\data-prep\setup.py", line 22, in <module>
    copyfile(src, dst)
  File "D:\anaconda3\envs\data-prep\lib\shutil.py", line 265, in copyfile
    with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
FileNotFoundError: [Errno 2] No such file or directory: 'D:\\anaconda3\\envs\\data-prep\\bin\\alos2_download_mosaic.py'


## 3) Prepare UTM tiles for AOI ##

### prep_tiles.py ###

### Usage ###

```
(data-prep) % prep_tiles.py [-h] (--aoishp aoishp | --admshp admshp) aoi_name tile_size
```

### Description ###

Prepare boundary and tiles for AOI.

### Positional Arguments ###

  **aoi_name**

    name of area of interest (AOI)

  **tile_size**        

    tile size in meters

### Optional Arguments ###

  **--aoishp**
  
    aoishp  shp/geojson of AOI

  **--admshp** 

    admshp  shp/geojson of subnational administrative boundaries
    
Create tiles for the Ucayali region of Peru with a tile size of 100 meters:


NOTE: what's the difference between aoishp and admshp?

In [ ]:
!python Utils/prep_tiles.py --aoishp AOI/ucayali/ucayali_boundary.geojson ucayali_tiles_1.geojson 100

## 4) Prepare Sentinel-1 Tiles ##

## Search Sentinel-1 granules on [ASF Vertex](https://search.asf.alaska.edu/#/) ##

1. Sign in using your Earthdata credentials. If you haven't used ASF Vertex before, you will need to agree their terms in order to use their HyP3 processing.

2. Use following "Additional Filters" when searching granules for your AOI:

    * File Type: L1 Detected High-Res Dual-Pol (GRD-HD)
    * Beam Mode: IW
    * Polarization: VV+VH

    ![vertex_search_filters](img/vertex_search_filters.png)

3. Add selected granules into download queue:

    ![vertex_add_queue](img/vertex_add_queue.png)

4. Download metadata files. At least download one csv or geojson file, which will be used for submitting HyP3 jobs.

    ![vertex_download_metadata](img/vertex_download_metadata.png)

5. Clear the selected granules in the downloads. Do not download these GRD-HD products as we will submit HyP3 jobs to apply radiometric terrain correction (RTC) to them.

## Submit HyP3 RTC jobs ##

### s1_submit_hyp3_jobs.py ###

### Usage ###

```
(data-prep) % s1_submit_hyp3_jobs.py [-h] [--dst dstpath] csv/geojson
```

### Description ###
    
Submit ASF HyP3 RTC processing jobs for Sentinel-1 granules.

Optionally copy or download the processed granules to the following destinations:

* AWS S3 bucket - s3://bucket/prefix

* Google Cloud Storage (GCS) - gs://bucket/prefix

* Local storage

### Positional Arguments ###

  **csv/geojson**
  
      Metadata file (csv or geojson) downloaded from ASF Vertex after data search.
  
### Optional Arguments ###

  **--dst dstpath**
  
      Destination path to store the processed granules.
      Supported dstpaths: AWS S3 - s3://your_bucket/some_prefix
                       GCS - gs://your_bucket/some_prefix
                       Local storage - your_local_path
      If dstpath not provided, AWS S3 buckets/keys for the processed granules will be displayed at the end.
      The processed granules will be saved in the following directory structure:
          dstpath
           └──year
               └──path_frame
                   └──processed_granules

### Notes ###

* Since ASF HyP3 stores the processed granules in their AWS S3 buckets, the data transfer will be much faster if you set up your S3 bucket to host these data. That is, using **s3://your_bucket/some_prefix** for --dst option.

* To set up AWS Command Line Interface (CLI) configurations and credentials (required if your dstpath is S3 or GCS):

    ```
    (data-prep) % aws configure
    ```

    where you will be asked to enter your **aws_access_key_id** and **aws_secret_access_key**.

* To set up Google Cloud gsutil tool (required if your dstpath is GCS):

    ```
    (data-prep) % gsutil config
    ```

    Then you will be prompted to sign in using your Google credentials. 

### Examples ###

* Run without --dst option

```
(data-prep) % s1_submit_hyp3_jobs.py 2017_94_215.geojson
Destination path for processed granules not provided. The download links will be listed at the end.

Enter Earthdata Username: xxxxxx
Enter Earthdata Password: 

Your remaining quota for HyP3 jobs: 977 granules.

You will be submitting the following granules for HyP3 RTC processing:
    2017_94_215 - 23 granules

Enter 'Y' to confirm you would like to submit these granules, or 'N' if you have already submitted the granules and want to copy the processed granules to your dstpath: N

Your processed granules for year_path_frame 2017_94_215 are available here:

hyp3-contentbucket-abcdefghijkl/12345abc-wxyz-1234-5678-opqrstuvwxyz/S1B_IW_20170111T032030_DVP_RTC30_G_gpunem_C229.zip
Expiration Time: 2021-09-18 00:00:00+00:00

...
...

Done with everything.
```

* Run with --dst option pointed to a S3 bucket

```
(data-prep) % s1_submit_hyp3_jobs.py 2017_94_215.geojson --dst s3://servir-public/test

Enter Earthdata Username: xxxxxx
Enter Earthdata Password: 

Your remaining quota for HyP3 jobs: 977 granules.

You will be submitting the following granules for HyP3 RTC processing:
    2017_94_215 - 23 granules

Enter 'Y' to confirm you would like to submit these granules, or 'N' if you have already submitted the granules and want to copy the processed granules to your dstpath: N

2017_94_215: copying processed granules to s3://servir-public/test
2017_94_215: DONE copying processed granules to s3://servir-public/test

Done with everything.
```


In [ ]:
!python Sentinel/s1_submit_hyp3_jobs.py . Sentinel/granules/ucayali/ucayali_sentinel_granules_2017.geojson

## Sentinel-1 Processing ##

### s1_proc.py ###

### Usage ###

```
(data-prep) % s1_proc.py [-h] [--pf path_frame] [--m1 m1] [--m2 m2] srcpath year
```

### Description ###
    
Processing Sentinel-1 RTC data, which includes two steps:

1. Calculating temporal mean for the Sentinel-1 granules acquired between the months \[m1, m2\]. This is done by **s1_build_vrt.py** and **calc_vrt_stats.py**.

2. Removing left/right (cross-track) edge pixels where the border noise is prominent. This is done by **remove_edges.py**.

### Positional Arguments ###

  **srcpath**
  
      Source path to where processed granules are stored.
      Supported dstpaths: AWS S3 - s3://your_bucket/some_prefix
                          GCS - gs://your_bucket/some_prefix
                          Local storage - your_local_path
  
  **year**
      
      Year of granules to be processed.

  
### Optional Arguments ###

  **--pf path_frame**
  
      'path_frame' of granules to be processed.
      If path_frame not provided, all path_frame's under srcpath/year will be processed.
  
  **--m1 m1**
  
      Granules with acquisition month >= m1 will be processed
  
  **--m2 m2**
  
      Granules with acquisition month <= m2 will be processed

### Notes ###

* The processing will be slow if srcpath is on AWS S3 or GCS because it requires heavy network I/O between the cloud and your local machine. If srcpath is on AWS S3, it is strongly recommended that you run the processing on AWS EC2.

In [ ]:
# sample run here
# print out vrts that are generated?

## 5) Prepare ALOS-2 tiles ##

## Download ALOS/ALOS-2 Mosaic ##

### alos2_download_mosaic.py ###

### Usage ###

```
(data-prep) % alos2_download_mosaic.py [-h] aoi year dst
```

### Description ###

Download ALOS/ALOS-2 Mosaic data from JAXA website.

### Positional Arguments ###

  **aoi**

      shp/geojson of area of interest (AOI)

  **year**

      Year

  **dst**

      Destination location (s3:// or gs:// or local paths). Downloaded data will be stored under dst/year/tarfiles/

### Notes ###

* Downloading ALOS/ALOS-2 Mosaic data requires a JAXA account, which can be registered from: https://www.eorc.jaxa.jp/ALOS/en/palsar_fnf/registration.htm.

In [ ]:
# sample run here

## ALOS-2 Processing ##

### alos2_proc.py ###

### Usage ###

```
(data-prep) % alos2_proc.py [-h] proj_dir aoi year
```

### Description ###

Process ALOS-2 tiles by appyling an Enhanced Lee Filter (why? is that all it does?)

### Positional Arguments ###

  **proj_dir**

      project directory (s3:// or gs:// or local dir). ALOS/ALOS-2 mosaic data (.tar.gz) will be stored under
      proj_dir/alos2_mosaic/year/tarfiles/

  **aoi**

      shp/geojson of area of interest (AOI)

  **year**

      year

### Notes ###

* ...

In [ ]:
# sample run here

## 6) Prepare Landsat Tiles ##

## Export Landsat NDVI ##

### gee_export_landsat_ndvi.py ###

### Usage ###

```
(data-prep) % gee_export_landsat_ndvi.py [-h] sitename tiles res year
```

### Description ###

Submit GEE processing for Landsat NDVI. 

### Positional Arguments ###

  **sitename**

      sitename 

  **tiles**

      shp/geojson file containing tiles onto which output raster will be resampled

  **res**

      Resolution
      
  **year**

      Year

In [ ]:
# sample run here

## 7) Prepare MODIS Tree Cover Tiles ##

## Export MODIS TC ##

### gee_export_modis_tc.py ###

### Usage ###

```
(data-prep) % gee_export_modis_tc.py [-h] sitename tiles res year
```

### Description ###

Submit GEE processing for MODIS tree cover

### Positional Arguments ###

  **sitename**

      sitename 

  **tiles**

      shp/geojson file containing tiles onto which output raster will be resampled

  **res**

      Resolution
      
  **year**

      Year. 

In [ ]:
# sample run here

## 8) Build Stacks ##

### build_stacks.py ###

### Usage ###

```
(data-prep) % build_stacks.py [-h] [--sitename sitename] proj_dir tiles year
```

### Description ###

Build 8 band stacks that include (C-VV / C-VH / C-INC / L-HH / L-HV / L-INC / NDVI / TC)
   
### Positional Arguments ###

  **proj_dir**
   
      project directory (s3:// or gs:// or local dirs)
  
  **tiles**
  
      shp/geojson file that contains tiles for the output stacks
      
  **year** 
  
      Year
      
###  Optional Arguements ###
      
  **sitename**
  
      sitename. If sitename not specified, proj_dir basename is used at sitename. 
  
### Notes ###

* ...

In [ ]:
# sample run here